in this notebook, I'm doing some work to pull the stats out for specific games for the nfl throughout the year. These were all one string in the table I scraped from RotoGuru1.com, so this table pulls the information and then cleans it up into something I can use.

Here is the page: http://rotoguru1.com/cgi-bin/fyday.pl?game=fd

In [1]:
#pulling in all the necessary python libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from tabulate import tabulate
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler

In [2]:
##realized the url just updated based on week number, so wrote a loop to accumulate all of the urls
weeks = list(range(1,4,1))
##every week added we just need to update this range

urls = []
for week in weeks:
    urls.append(str('http://rotoguru1.com/cgi-bin/fyday.pl?week='+str(week)+'&game=fd'))

##we should now have the page we need for each week

This is the script to scrape all of the player data. I only ran this once and saved it as a csv in the section in the code marked below because running it is very time consuming.

In [3]:
##creating an empty list that will have the player row for each table

rows = []

##Running a loop through all the urls to get every piece of data in one list
##scraping each url
for url in urls:
    webpage = requests.get(url)
    webpage_content = webpage.content
    soup = BeautifulSoup(webpage_content,'html.parser')
    table_rows = soup.find_all('td')
    #pulling just the player data, which starts on the 20th entry. 
    #We also want to add some kind of date identifier so for now im doing the url since it has the week in it
    for row in table_rows[20:]:
        rows.append([row.get_text(),url])

This is where I saved what I scraped as a csv.

In [4]:
##create data frame
sample_frame = pd.DataFrame.from_records(rows).reset_index()
##save to csv
sample_frame.to_csv('All_nfl_rows.csv')

The above text can be uncommented if you ever need to pull the information again, for example in week 3. The below information is now pulling from the "All_nfl_rows" CSV I saved, which has all data from each week on my local and is regularly updated using the above script.<br><br>
From here on, I'm going to approach the problem using that static csv, starting with creating a data frame

In [5]:
#create frame from saved csv
new_frame = pd.read_csv('All_nfl_rows.csv')
#rename columns
new_frame.columns = ['Row','ID','Data','URL']

In [6]:
#clean up column with urls so it just has date
dates = new_frame['URL'].map(lambda x: x.replace('http://rotoguru1.com/cgi-bin/fyday.pl?week=','')\
                             .replace('&game=fd',''))
new_frame['Week'] = dates

#create separate frame that removes all the url columns
date_frame = new_frame[['Data','Week']].reset_index(drop=True)

Everything from here to the next markup is me clearing out rows that were in the table but did not have player information.

In [7]:
#We need to remove some ads and nav stuff here by 
#converting to a series, finding the ones that match, and adding back to the table
find_Ads = date_frame['Data']
#create series that has 0 for what matches the ads
ads_found = find_Ads.str.find('RotoGuru')
#add column to table with 0's
date_frame['Ad'] = ads_found
#create new table with those rows with zero gone
no_ads = date_frame[date_frame['Ad'] != 1].reset_index(drop=True)
len(no_ads)

7098

In [8]:
#Repeating to remove Jump To:
#converting to a series, finding the ones that match, and adding back to the table
find_jump = no_ads['Data']
#create series that has 0 for what matches the text
jump_found = find_jump.str.find('Jump to:')
#add column to table with 0's
no_ads['Remove'] = jump_found
#create new table with those rows with zero gone
jump_gone = no_ads[no_ads['Remove'] !=0].reset_index(drop=True)

In [9]:
#There's a term 'Unlisted' that pops up occasionally and breaks everything in the NBA version of this page.
#I'm clearing that here to be safe
find_unlisted = jump_gone['Data']
#create series that has a 0 for where it says unlisted
unlisted_found = find_unlisted.str.find('Unlisted')
#add column to table with 0's
jump_gone['Z'] = unlisted_found
#create new table with those rows removed
unlisted_gone = jump_gone[jump_gone['Z'] != 0].reset_index(drop=True)

There's a few headers in our table that weren't in the nba one. I'm clearing those out here.

In [10]:
#repeating for Opp 
find_opp = unlisted_gone['Data']
#create series that has a 0 for where it says unlisted
opp_found = find_opp.str.find('Opp.')
#add column to table with 0's
unlisted_gone['F'] = opp_found
#create new table with those rows removed
opp_gone = unlisted_gone[unlisted_gone['F'] != 0].reset_index(drop=True)
#Repeating for Wide Recievers
find_wide = opp_gone['Data']
#create series that has a 0 for where it says unlisted
wide_found = find_wide.str.find('Wide Receivers')
#add column to table with 0's
opp_gone['X'] = wide_found
#create new table with those rows removed
wide_gone = opp_gone[opp_gone['X'] != 0].reset_index(drop=True)
#Repeating for \n
find_n = wide_gone['Data']
##create series that has a 0 for where it says unlisted
n_found = find_n.str.find('\n\n\n\n')
##add column to table with 0's
wide_gone['W'] = n_found
##create new table with those rows removed
n_gone = wide_gone[wide_gone['W'] != 0].reset_index(drop=True)
##Repeating for Score
find_score = n_gone['Data']
##create series that has a 0 for where it says unlisted
score_found = find_n.str.find('Score')
##add column to table with 0's
n_gone['R'] = score_found
##create new table with those rows removed
score_gone = n_gone[n_gone['R'] != 0].reset_index(drop=True)
##Repeating for Tight Ends
find_tight = score_gone['Data']
##create series that has a 0 for where it says unlisted
tight_found = find_tight.str.find('Tight Ends')
##add column to table with 0's
score_gone['S'] = tight_found
##create new table with those rows removed
tight_gone = score_gone[score_gone['S'] != 0].reset_index(drop=True)

There's a few other terms that kept popping up - I ended up finding a more efficient way to clear them out but have left both methods in this notebook so I can see the two different options. str.contains() would be a little more dangerous for a two letter arrangement for common than QB, especially if it wasn't uppercase.

In [11]:
##There's a subtable headers that aren't player data. we are getting rid of most those here.
##creates list of all the words I want to find and get rid of
sub = ['QB','Points','Team','Salary','Unlisted','Running Backs',\
       'Kickers','Defenses']
pattern = '|'.join(sub)

tight_gone['gone'] = tight_gone['Data'].str.contains(pattern, case=True)

In [12]:
#remove any rows where we found those subtable headers
clean_table = tight_gone[tight_gone['gone'] != True].reset_index(drop=True)

In [13]:
##create series with the data
just_data = clean_table[['Data','Week']].reset_index(drop=True)

In [14]:
#merging data and date in a column so
#I can then hopefully turn each one into a series and then just have the date once at the end.
just_data['merge_date'] = just_data['Data'].astype(str)+'|'+just_data['Week']

In [15]:
#turning my merged column into a list so I can run a comprehension and then add the date to the end of a player row
just_datas = just_data['merge_date']

In [16]:
#turning series into a list so we can do some stuff
data_list = list(just_datas)

In [17]:
#I used this loop to create sublists per player based on each player entry having 5 columns
##each row was 5 entries. This gets thrown off very easily though so we need to be careful to remove all other data
##which we have already done above
players = [data_list[x:x+5] for x in range(0, len(data_list), 5)]

In [18]:
#now we have a row per player, but every piece of data has the date. 
#So first I'm pulling the date out and adding it to the end of every sublist
for player in players:
    player.insert(0,player[0].split('|')[1])

In [19]:
#creating list that has each player entry as its own record without date. 
#Note - we already pulled the date and added it to the end of the sublist.
#If you haven't done that I recommend you do so first
player_rows =[]
for player in players:
    if len(player) == 6:
        player_rows.append([player[0],\
                            player[1].split('|')[0],\
                            player[2].split('|')[0],\
                            player[3].split('|')[0],\
                            player[4].split('|')[0],\
                            player[5].split('|')[0],\
                            ])


In [20]:
sample_frame = pd.DataFrame.from_records(player_rows).reset_index(drop=True)

In [21]:
sample_frame.columns = ['Week','Name','Team','Opponent','Fanduel_Points','Fanduel_Price']

In [22]:
#Replace currency symbols in column so we can make it an integer
sample_frame['Fanduel_Price'] = sample_frame['Fanduel_Price'].str.replace(',', '')
sample_frame['Fanduel_Price'] = sample_frame['Fanduel_Price'].str.replace('$', '')
#Turn the column to integers
sample_frame['Fanduel_Price'] = pd.to_numeric(sample_frame['Fanduel_Price'], errors='coerce')

Now that I've got a frame with consistent columns, I'm going to clean things up by making some columns a float and also splitting out the opponent so we can see home vs away.

In [23]:
#there is no price listed for some players. i am removing them since the whole goal is to see who exceeds their price
df1 = sample_frame[~sample_frame['Fanduel_Price'].isna()].reset_index(drop=True)
#Convert Fanduel_Points to float
df1['Fanduel_Points'] = pd.to_numeric(df1['Fanduel_Points'])
#create column for home vs away and updated column for opponent
df1['Split'] = df1['Opponent'].str[0]
df1['Home'] = df1.Split.apply(lambda x: 'Home' if str(x) == 'v' else 'Away')
df1['Foe'] = df1['Opponent'].str[1:]
#get rid of periods in home v away
df1['Opponent'] = df1['Foe'].str.replace('. ','')
#get rid of some carrots that are appearing
df1['Name'] = df1['Name'].str.replace('^','')

In [24]:
#removing Split column
del df1['Split']
del df1['Foe']

I now have a frame with the data for all players week to week. I should now be able to refresh this information every week. as long as I scrape the new scores first.

In [25]:
df1

,Week,Name,Team,Opponent,Fanduel_Points,Fanduel_Price,Home
0,1,"Wilson, Russell",sea,atl,31.78,8400.0,Away
1,1,"Rodgers, Aaron",gnb,min,30.76,7600.0,Away
2,1,"Allen, Josh",buf,nyj,28.18,7900.0,Home
3,1,"Jackson, Lamar",bal,cle,27.50,9300.0,Home
4,1,"Murray, Kyler",ari,sfo,27.30,7700.0,Away
...,...,...,...,...,...,...,...
1297,3,Washington,was,cle,1.00,3900.0,Away
1298,3,Jacksonville,jac,mia,0.00,3800.0,Home
1299,3,New York J,nyj,ind,0.00,3800.0,Away
1300,3,New York G,nyg,sfo,-2.00,3600.0,Home


In [27]:
#going ahead and saving this for a safe template
df1.to_csv('nfl_weekly_fanduel_scores.csv')

In the next notebook ("Manipulating Fanduel football"), I'm going to try to add some columns to this data, specifically position.